In [2]:
import numpy as np
import pandas as pd
import datetime as dt
import pickle
import re
import unidecode
import math as m

In [22]:
def preprocessing(text):

  #print(text)
  try: #text == None:
    #pass
  #else:
# Remove all the special characters
    text = re.sub('\W+',' ', text)
 
# Substituing all numnbers
    text = re.sub('[0-9]', '', text)

# Remove all single characters
    text = re.sub(r"\b[a-zA-Z]\b", "", text)

# Remove all single characters
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)

# Remove single characters from the start
    text = re.sub(r'\^[a-zA-Z]\s+', ' ', text)

    text = text.replace('_',' ')

# Converting to Lowercase
    text = text.lower()

# Remove accents
    text = unidecode.unidecode(text)

# Remove words with less or equal than 2 letters
    text = re.sub(r'\b\w{1,2}\b', '', text)

# Substituting multiple spaces with single space
    text = re.sub(r'\s+', ' ', text, flags=re.I)

# Remove initial and final spaces  
    text = text.strip()

# Lemmatization
#    wnl = WordNetLemmatizer()
#    text = text.split()
#    text = [wnl.lemmatize(word) for word in text]
#    text = [word for word in text if word not in en_stop]
#    text = [word for word in text if len(word) > 3]
#    text = ' '.join(text)
    return text
  except:
    pass

def preprocessing_2(text):
  try:
    text = text.replace('MLB-','')
    text = text.replace('_',' ')
    text = text.lower()
    return text
  except:
    pass

def preprocessing_3(text):
  try:
    text = str(text)
    text = text.replace('0','zero')
    text = text.replace('1','um')
    text = text.replace('2','dois')
    text = text.replace('3','treis')
    text = text.replace('4','quatro')
    text = text.replace('5','cinco')
    text = text.replace('6','seis')
    text = text.replace('7','sete')
    text = text.replace('8','oito')
    text = text.replace('9','nove')
    text = text.replace('MLB','')
    text = text.strip()
    return text
  except:
    pass

def preprocessing_4(text):
  try:
    text = re.sub('\W+',' ', text)
    text = text.replace('0','zero')
    text = text.replace('1','um')
    text = text.replace('2','dois')
    text = text.replace('3','treis')
    text = text.replace('4','quatro')
    text = text.replace('5','cinco')
    text = text.replace('6','seis')
    text = text.replace('7','sete')
    text = text.replace('8','oito')
    text = text.replace('9','nove')
    text = unidecode.unidecode(text)
    text = re.sub(r'\s+', ' ', text, flags=re.I)  
    text = text.strip()
    text = text.lower()
    return text
  except:
    pass

def preprocessing_5(text):
  try:
    text = str(text)
    text = text.replace('0','a')
    text = text.replace('1','b')
    text = text.replace('2','c')
    text = text.replace('3','d')
    text = text.replace('4','e')
    text = text.replace('5','f')
    text = text.replace('6','g')
    text = text.replace('7','h')
    text = text.replace('8','i')
    text = text.replace('9','j')
    text = text.replace('MLB','')
    text = unidecode.unidecode(text)
    text = re.sub(r'\s+', ' ', text, flags=re.I)  
    text = text.strip()
    text = text.lower()
    return text
  except:
    pass

In [52]:
#Carga del Conjunto de Datos

ds = pd.read_pickle('ds.pkl')

In [53]:
ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2298887 entries, 0 to 2298886
Data columns (total 43 columns):
 #   Column               Dtype  
---  ------               -----  
 0   Unnamed: 0           int64  
 1   item_id              object 
 2   shp_service_id       float64
 3   comp_length          float64
 4   shp_quantity         float64
 5   shp_weight           float64
 6   shp_shipment_id      float64
 7   comp_weight          float64
 8   shp_width            float64
 9   comp_height          float64
 10  site_id              object 
 11  domain_id            object 
 12  attributes           object 
 13  shipping_mode        object 
 14  title                object 
 15  shp_length           float64
 16  model_name           object 
 17  category_id          object 
 18  condition            object 
 19  catalog_product_id   object 
 20  shp_date_created     object 
 21  logistic_type        object 
 22  permalink            object 
 23  shp_volume           float64
 24

In [54]:
# Generación de la etiqueta real de los productos
# 1: Maquinable - 0: No Maquinable

ds['gt'] = (ds['shp_length'] < 70) & (ds['shp_width'] < 70) & (ds['shp_height'] < 70)
ds['gt'] = ds['gt']*1

In [55]:
# Se eliminan los títulos duplicados para generar un conjunto de datos de productos
ds = ds.drop_duplicates(subset='title')

In [56]:
# Se genera la columna label dado que se necesitará para el entrenamiento de la librería Fast Text

mapping = {1:'__label__shorter_70cm', 0:'__label__longer_70cm'}
ds['label'] = ds['gt'].map(mapping)

# En base a análisis realizado y según la consulta del equipo exporte, se realizó una selección inicial de variables que puede utilizarse para construir la solución predictiva

ds = ds[['gt','title','label','domain_id','category_id','seller_id','brand_name','L1','L2','L3','shp_weight','price']]
#train = train.dropna()

In [57]:
# Muestra de títulos original
print(ds['title'].head(50))

0     Bau Tork 45 Litros + Bagageiro Titan Fan 125 2...
1     Protetor De Sofá Mariana 1 Lugar Dupla-face Po...
2     Tablet Powerpack Tela 10.1'' Ips Hd Android 7....
3     Capa Silicone Borracha Tablet Apple iPad Pro 1...
4     Silencioso Escapamento Motor Honda Gx240 - 7hp...
5     Abajur Lustre Quarto Luminaria De Mesa Menina ...
6     Lâmpada Led Globo De Luz Giratória  Rgb Dj Jog...
7            Kit Relação Completa Yamaha Xtz 125 Xtz125
8      Kit Relação Completa Velth Yamaha Xtz 125 Xtz125
9                Kit Hidratante Sabonete Overdose Racco
10                   Bandeja Civic 2001 Á 2005 Sem Pivô
11           Vinho Luigi Bosca Malbec Cx Com 6 Promoção
12    Lp Ghost  Opvs Eponymovs - Ed. Limitada + Fret...
13                   Vodka Ciroc Coconut 750ml Original
14    Capa Banco Carro 100% Couro Ecológico Universa...
15    Lindo Poncho Pelerine De Lã Com Pelos Moda Bou...
16    Pantovin Kit Crescimento Cabelos E Anti Queda ...
17      Sunga Em Lycra Juvenil -  Nº 14 ( G / Ju

In [58]:
#Ejecución del preprocesamiento sobre los descriptores textuales

ds['title'] = ds['title'].transform(lambda i: preprocessing(i))
ds['seller_id'] = ds['seller_id'].transform(lambda i: preprocessing_5(i))
ds['brand_name'] = ds['brand_name'].transform(lambda i: preprocessing_4(i))
ds['brand_name'] = ds['brand_name'].transform(lambda i: preprocessing(i))
ds['L1'] = ds['L1'].transform(lambda i: preprocessing(i))
ds['L2'] = ds['L2'].transform(lambda i: preprocessing(i))
ds['L3'] = ds['L3'].transform(lambda i: preprocessing(i))
ds = ds.drop_duplicates('title')
ds = ds.reset_index(drop=True)

In [59]:
# Realizar un chequeo sobre las valores nulos

ds.isna().sum()

gt                  0
title               1
label               0
domain_id        5888
category_id         1
seller_id           0
brand_name     172195
L1                  3
L2                  3
L3             228264
shp_weight          0
price               0
dtype: int64

In [60]:
# Eliminación del único título nulo

ds.dropna(subset=['title'], inplace=True)

In [61]:
# Este manejo de valores nulos se realiza para que no se produzcan errores en la fase de entrenamiento

ds['domain_id'] = ds['domain_id'].transform(lambda i: '' if i == None else i)
ds['category_id'] = ds['category_id'].transform(lambda i: '' if i == None else i)
ds['brand_name'] = ds['brand_name'].transform(lambda i: '' if i == None else i)
ds['L1'] = ds['L1'].transform(lambda i: '' if i == None else i)
ds['L2'] = ds['L2'].transform(lambda i: '' if i == None else i)
ds['L3'] = ds['L3'].transform(lambda i: '' if i == None else i)

In [62]:
# Realizar un chequeo sobre las valores nulos

ds.isna().sum()

gt             0
title          0
label          0
domain_id      0
category_id    0
seller_id      0
brand_name     0
L1             0
L2             0
L3             0
shp_weight     0
price          0
dtype: int64

In [35]:
# Muestra de títulos con preprocesamiento

print(ds['title'].head(50))

0        bau tork litros bagageiro titan fan preto poli
1     protetor sofa mariana lugar dupla face poltron...
2                tablet powerpack tela ips android chip
3     capa silicone borracha tablet apple ipad pro p...
4                    silencioso escapamento motor honda
5            abajur lustre quarto luminaria mesa menina
6       lampada led globo luz giratoria rgb jogo balada
7                   kit relacao completa yamaha xtz xtz
8             kit relacao completa velth yamaha xtz xtz
9                kit hidratante sabonete overdose racco
10                               bandeja civic sem pivo
11                vinho luigi bosca malbec com promocao
12           ghost opvs eponymovs limitada frete gratis
13                         vodka ciroc coconut original
14    capa banco carro couro ecologico universal tod...
15    lindo poncho pelerine com pelos moda boutique ...
16    pantovin kit crescimento cabelos anti queda co...
17                          sunga lycra juvenil 

In [63]:
#Se guarda el conjunto de datos preprocesado

ds.to_pickle('ds_pre.pkl')